# Tools for Game Theory

**Daisuke Oyama**  
*Faculty of Economics, University of Tokyo*

This notebook demonstrates the functionalities of the `Player` and `NormalFormGame` types
in [QuantEcon/Games.jl](https://github.com/QuantEcon/Games.jl).

The first time you run this notebook,
you need to install the Games.jl package by removing the "#" below:

In [ ]:
#Pkg.clone("https://github.com/QuantEcon/Games.jl")
#Pkg.add("Games")

In [ ]:
using Games
#using Compat  # For compatibility for Julia v0.4

## Normal Form Games

An $N$-player *normal form game* is a triplet $g = (I, (A_i)_{i \in I}, (u_i)_{i \in I})$ where

* $I = \{1, \ldots, N\}$ is the set of players,
* $A_i = \{1, \ldots, n_i\}$ is the set of actions of player $i \in I$, and
* $u_i \colon A_i \times A_{i+1} \times \cdots \times A_{i+N-1} \to \mathbb{R}$
  is the payoff function of player $i \in I$,

where $i+j$ is understood modulo $N$.

Note that we adopt the convention that the $1$-st argument of the payoff function $u_i$ is
player $i$'s own action
and the $j$-th argument, $j = 2, \ldots, N$, is player ($i+j-1$)'s action (modulo $N$).

In our package,
a normal form game and a player are represented by
the types `NormalFormGame` and `Player`, respectively.

A `Player` carries the player's payoff function and implements in particular
a method that returns the best response action(s) given an action of the opponent player,
or a profile of actions of the opponents if there are more than one.

A `NormalFormGame` is in effect a container of `Player` instances.

### Creating a `NormalFormGame`

There are several ways to create a `NormalFormGame` instance.

The first is to pass an array of payoffs for all the players, i.e.,
an $(N+1)$-dimenstional array of shape $(n_1, \ldots, n_N, N)$
whose $(a_1, \ldots, a_N)$-entry contains an array of the $N$ payoff values
for the action profile $(a_1, \ldots, a_N)$.

As an example, consider the following game ("**Matching Pennies**"):

$
\begin{bmatrix}
1, -1 & -1, 1 \\
-1, 1 & 1, -1
\end{bmatrix}
$

In [ ]:
matching_pennies_bimatrix = Array(Float64, 2, 2, 2)
matching_pennies_bimatrix[1, 1, :] = [1, -1]  # payoff profile for action profile (1, 1)
matching_pennies_bimatrix[1, 2, :] = [-1, 1]
matching_pennies_bimatrix[2, 1, :] = [-1, 1]
matching_pennies_bimatrix[2, 2, :] = [1, -1]
g_MP = NormalFormGame(matching_pennies_bimatrix)

In [ ]:
g_MP.players[1]  # Player instance for player 1

In [ ]:
g_MP.players[2]  # Player instance for player 2

In [ ]:
g_MP.players[1].payoff_array  # Player 1's payoff array

In [ ]:
g_MP.players[2].payoff_array  # Player 2's payoff array

In [ ]:
g_MP[1, 1]  # payoff profile for action profile (1, 1)

If a square matrix (2-dimensional array) is given,
then it is considered to be a symmetric two-player game.

Consider the following game (symmetric $2 \times 2$ "**Coordination Game**"):

$
\begin{bmatrix}
4, 4 & 0, 3 \\
3, 0 & 2, 2
\end{bmatrix}
$

In [ ]:
coordination_game_matrix = [4 0;
                            3 2]  # square matrix
g_Coo = NormalFormGame(coordination_game_matrix)

In [ ]:
g_Coo.players[1].payoff_array  # Player 1's payoff array

In [ ]:
g_Coo.players[2].payoff_array  # Player 2's payoff array

Another example ("**Rock-Paper-Scissors**"):

$
\begin{bmatrix}
 0,  0 & -1,  1 &  1, -1 \\
 1, -1 &  0,  0 & -1,  1 \\
-1,  1 &  1, -1 &  0,  0
\end{bmatrix}
$

In [ ]:
RPS_matrix = [0 -1 1;
              1 0 -1;
              -1 1 0]
g_RPS = NormalFormGame(RPS_matrix)

The second is to specify the sizes of the action sets of the players
to create a `NormalFormGame` instance filled with payoff zeros,
and then set the payoff values to each entry.

Let us construct the following game ("**Prisoners' Dilemma**"):

$
\begin{bmatrix}
1, 1  & -2, 3 \\
3, -2 & 0, 0
\end{bmatrix}
$

In [ ]:
g_PD = NormalFormGame((2, 2))  # There are 2 players, each of whom has 2 actions
g_PD[1, 1] = [1, 1]
g_PD[1, 2] = [-2, 3]
g_PD[2, 1] = [3, -2]
g_PD[2, 2] = [0, 0];

In [ ]:
g_PD

In [ ]:
g_PD.players[1].payoff_array

Finally, a `NormalFormGame` instance can be constructed by giving an array of `Player` instances,
as explained in the next section.

### Creating a `Player`

A `Player` instance is created by passing an array of dimension $N$
that represents the player's payoff function ("payoff array").

Consider the following game (a variant of "**Battle of the Sexes**"):

$
\begin{bmatrix}
3, 2 & 1, 1 \\
0, 0 & 2, 3
\end{bmatrix}
$

In [ ]:
player1 = Player([3 1; 0 2])
player2 = Player([2 0; 1 3]);

Beware that in `payoff_array[h, k]`, `h` refers to the player's own action,
while `k` refers to the opponent player's action.

In [ ]:
player1.payoff_array

In [ ]:
player2.payoff_array

Passing an array of Player instances is the third way to create a `NormalFormGame` instance:

In [ ]:
g_BoS = NormalFormGame((player1, player2))

### More than two players

Games with more than two players are also supported.

Let us consider the following version of $N$-player **Cournot Game**.

There are $N$ firms (players) which produce a homogeneous good
with common constant marginal cost $c \geq 0$.
Each firm $i$ simultaneously determines the quantity $q_i \geq 0$ (action) of the good to produce.
The inverse demand function is given by the linear function $P(Q) = a - Q$, $a > 0$,
where $Q = q_1 + \cdots + q_N$ is the aggregate supply.
Then the profit (payoff) for firm $i$ is given by
$$
u_i(q_i, q_{i+1}, \ldots, q_{i+N-1})
= P(Q) q_i - c q_i
= \left(a - c - \sum_{j \neq i} q_j - q_i\right) q_i.
$$

Theoretically, the set of actions, i.e., available quantities, may be
the set of all nonnegative real numbers $\mathbb{R}_+$
(or a bounded interval $[0, \bar{q}]$ with some upper bound $\bar{q}$),
but for computation on a computer we have to discretize the action space
and only allow for finitely many grid points.

The following script creates a `NormalFormGame` instance of the Cournot game as described above,
assuming that the (common) grid of possible quantity values is stored in an array `q_grid`.

In [ ]:
immutable Cournot{N} end

@compat function (::Type{Cournot{N}}){N,T<:Real}(a::Real, c::Real, q_grid::Vector{T})
    nums_actions = tuple([length(q_grid) for i in 1:N]...)::NTuple{N,Int}
    S = promote_type(typeof(a), typeof(c), T)
    payoff_array= Array{T}(nums_actions)
    for I in CartesianRange(nums_actions)
        Q = zero(S)
        for i in 1:N
            Q += q_grid[I[i]]::T
        end
        payoff_array[I] = (a - c - Q) * q_grid[I[1]]
    end
    players = tuple([Player(payoff_array) for i in 1:N]...)::NTuple{N,Player{N,T}}
    return NormalFormGame(players)
end

cournot{T<:Real}(a::Real, c::Real, N::Integer, q_grid::Vector{T}) =
    Cournot{N}(a, c, q_grid)

Here's a simple example with three firms,
marginal cost $20$, and inverse demand function $80 - Q$,
where the feasible quantity values are assumed to be $10$ and $15$.

In [ ]:
a, c = 80, 20
N = 3
q_grid = [10, 15]  # [1/3 of Monopoly quantity, Nash equilibrium quantity]

g_Cou = cournot(a, c, N, q_grid)

In [ ]:
print(g_Cou.players[1])

In [ ]:
g_Cou.nums_actions

## Nash Equilibrium

A *Nash equilibrium* of a normal form game is a profile of actions
where the action of each player is a best response to the others'.

The `Player` object has methods `best_response` and `best_responses`.

Consider the Matching Pennies game `g_MP` defined above.
For example, player 1's best response to the opponent's action 2 is:

In [ ]:
best_response(g_MP.players[1], 2)

Player 1's best responses to the opponent's mixed action `[0.5, 0.5]`
(we know they are 1 and 2):

In [ ]:
# By default, returns the best response action with the smallest index
best_response(g_MP.players[1], [0.5, 0.5])

In [ ]:
# With tie_breaking='random', returns randomly one of the best responses
best_response(g_MP.players[1], [0.5, 0.5], tie_breaking="random")  # Try several times

`best_responses` returns an array of all the best responses:

In [ ]:
best_responses(g_MP.players[1], [0.5, 0.5])

For this game, we know that `([0.5, 0.5], [0.5, 0.5])` is a (unique) Nash equilibrium.

In [ ]:
is_nash(g_MP, ([0.5, 0.5], [0.5, 0.5]))

In [ ]:
is_nash(g_MP, (1, 1))

In [ ]:
is_nash(g_MP, ([1., 0.], [0.5, 0.5]))

### Finding Nash equilibria

Our package does not have sophisticated algorithms to compute Nash equilibria (yet)...
One might look at [Gambit](http://gambit.sourceforge.net),
which implements several such algorithms.

#### Brute force

For small games, we can find pure action Nash equilibria by brute force,
by calling the method `pure_nash`.

In [ ]:
function print_pure_nash_brute(g::NormalFormGame)
    NEs = pure_nash(g)
    num_NEs = length(NEs)
    if num_NEs == 0
        msg = "no pure Nash equilibrium"
    elseif num_NEs == 1
        msg = "1 pure Nash equilibrium:\n$(NEs[1])"
    else
        msg = "$num_NEs pure Nash equilibria:\n"
        for (i, NE) in enumerate(NEs)
            i < num_NEs ? msg *= "$NE," : msg *= "$NE"
        end
    end
    println(join(["The game has ", msg]))
end

Matching Pennies:

In [ ]:
print_pure_nash_brute(g_MP)

Coordination game:

In [ ]:
print_pure_nash_brute(g_Coo)

Rock-Paper-Scissors:

In [ ]:
print_pure_nash_brute(g_RPS)

Battle of the Sexes:

In [ ]:
print_pure_nash_brute(g_BoS)

Prisoners' Dillema:

In [ ]:
print_pure_nash_brute(g_PD)

Cournot game:

In [ ]:
print_pure_nash_brute(g_Cou)

#### Sequential best response

In some games, such as "supermodular games" and "potential games",
the process of sequential best responses converges to a Nash equilibrium.

Here's a script to find *one* pure Nash equilibrium by sequential best response, if it converges.

In [ ]:
function sequential_best_response(g::NormalFormGame;
                                  init_actions::Union{Vector{Int},Void}=nothing,
                                  tie_breaking="smallest",
                                  verbose=true)
    N = num_players(g)
    a = Array{Int}(N)
    if init_actions == nothing
        init_actions = ones(Int, N)
    end
    copy!(a, init_actions)
    if verbose
        println("init_actions: $a")
    end
    
    new_a = Array{Int}(N)
    max_iter = prod(g.nums_actions)
    
    for t in 1:max_iter
        copy!(new_a, a)
        for (i, player) in enumerate(g.players)
            if N == 2
                a_except_i = new_a[3-i]
            else
                a_except_i = (new_a[i+1:N]..., new_a[1:i-1]...)
            end
            new_a[i] = best_response(player, a_except_i,
                                     tie_breaking=tie_breaking)
            if verbose
                println("player $i: $new_a")
            end
        end
        if new_a == a
            return a
        else
            copy!(a, new_a)
        end
    end
    
    println("No pure Nash equilibrium found")
    return a
end

A Cournot game with linear demand is known to be a potential game,
for which sequential best response converges to a Nash equilibrium.

Let us try a bigger instance:

In [ ]:
a, c = 80, 20
N = 3
q_grid = collect(linspace(0, a-c, 13))  # [0, 5, 10, ..., 60]
g_Cou = cournot(a, c, N, q_grid)

In [ ]:
a_star = sequential_best_response(g_Cou)  # By default, start with (1, 1, 1)
println("Nash equilibrium indices: $a_star")
println("Nash equilibrium quantities: $(q_grid[a_star])")

In [ ]:
# Start with the largest actions (13, 13, 13)
sequential_best_response(g_Cou, init_actions=[13, 13, 13])

The limit action profile is indeed a Nash equilibrium:

In [ ]:
is_nash(g_Cou, tuple(a_star...))

In fact, the game has other Nash equilibria
(because of our choice of grid points and parameter values):

In [ ]:
print_pure_nash_brute(g_Cou)

Make it bigger:

In [ ]:
N = 4
q_grid = collect(linspace(0, a-c, 61))  # [0, 1, 2, ..., 60]
g_Cou = cournot(a, c, N, q_grid)

In [ ]:
sequential_best_response(g_Cou)

In [ ]:
sequential_best_response(g_Cou, init_actions=[1, 1, 1, 31])

Sequential best response does not converge in all games:

In [ ]:
print(g_MP)  # Matching Pennies

In [ ]:
sequential_best_response(g_MP)